# Analisando os dados disponibilizados

### 1) Importar bibliotecas

In [ ]:
import pandas as pd
from unidecode import unidecode
import numpy as np

### 2) Ler dados brutos

In [ ]:
escolas = pd.read_csv("dados_brutos/escolas.csv")

material_didatico = pd.read_csv("dados_brutos/material_didatico.csv")

subprefeituras = pd.read_csv("dados_brutos/subprefeituras.csv")

### 3) Criar cópias dos DFs

In [ ]:
subprefeituras_copia = subprefeituras.copy()

In [ ]:
material_didatico_copia = material_didatico.copy()

In [ ]:
escolas_copia = escolas.copy()

### 4) Tratamento dos DFs

#### 4.1) Tratamento geral (explícito no arquivo .md)

1) aplicar lower para todas as colunas
2) nome das colunas em snake_case
3) strings não devem conter acentos
4) todas as strings devem estar em maiúsculo

In [ ]:
def renomear_colunas(df):
    new_columns = {col: unidecode(col.lower().strip()) for col in df.columns}
    df.rename(columns=new_columns, inplace=True)
    for col in df.columns:
        df[col] = df[col].apply(lambda x: unidecode(str(x)).upper()) if df[col].dtypes == 'object' else df[col]

In [ ]:
# lista dfs
dfs = [escolas_copia, material_didatico_copia, subprefeituras_copia]

# trata cada df
for df in dfs:
    renomear_colunas(df)
    display(df.head(5))

#### 4.2) Tratamento geral (não exposto no arquivo .md)

1) Remove valores duplicados
2) Substitui valores nulos

In [ ]:
# lista dfs
dfs = [escolas_copia, material_didatico_copia, subprefeituras_copia]

# trata cada df
for df in dfs:
    # remove duplicados
    df.drop_duplicates(inplace=True)
    if 'id' in df.columns:
        df.drop_duplicates(subset='id', keep='first', inplace=True)
    
    # substitui valores nulos do campos numéricos
    for col in df.columns:
        if df[col].dtypes == 'int' or df[col].dtypes == 'float':
            df[col] = df[col].fillna(0)

### 4.3) Tratamento para o DF material didático


In [ ]:
material_didatico_copia.info()

#### O que deve ser feito:

1. Converter tipo da coluna quantidade de objeto para inteiro
2. Substituir qualquer valor que não seja numérico por um número na coluna quantidade

#### Verifica valores diferentes de números

In [ ]:
for i, row in material_didatico_copia.iterrows():
    qnt = row['quantidade']
    if not qnt.isdigit():
        print(qnt)

#### Converte todos os valores da coluna para int

In [ ]:
material_didatico_copia['quantidade'] = material_didatico_copia['quantidade'].apply(lambda x: int(x) if x.isdigit() else 0)

Filtra dataframe com quantidade de material diferente de zero

In [ ]:
material_didatico_copia = material_didatico_copia[material_didatico_copia['quantidade'] != 0]

#### Verifica existência de valores nulos, duplicados e existência de valores iguais a 0

In [ ]:
print("Quantidade de valores nulos:", material_didatico_copia.isna().sum().tolist()[0])
print("Quantidade de valores duplicados:", material_didatico_copia.duplicated().sum())
print("Quantidade de valores iguais a 0 (coluna quantidade):", len(material_didatico_copia[material_didatico_copia['quantidade'] == 0]))

### 4.3) Tratamento para o DF subprefeituras

#### Verifica existência de valores nulos e duplicados

In [ ]:
print("Quantidade de valores nulos:", subprefeituras_copia.isna().sum().tolist()[0])
print("Quantidade de valores duplicados:", subprefeituras_copia.duplicated().sum())

### 4.4) Tratamento para o DF escola

In [ ]:
escolas_copia.info()

#### O que deve ser feito:

1. Converter tipo das colunas lat e lon de objeto para float
2. Criar campos logradouro e numero a partir do campo endereco
3. Criar campo tipo escola a partir do campo que contém nome das escolas (escolas_postos)
4. Merge entre os DFs (escola, material didático e subprefeituras)
5. Escrever arquivos csv

#### Converter tipo das colunas lat e lon de objeto para float

1) Substitui vírgula por ponto para validar coordenadas (requisito para analisar números float)

In [ ]:
# converte vŕgula em ponto nos campos lat e lon
escolas_copia[['lat', 'lon']] = escolas_copia[['lat', 'lon']].applymap(lambda x: x.replace(",", "."))

2) Valida coordenadas lat e lon

In [ ]:
for i, linha in escolas_copia.iterrows():
    lat = linha['lat']
    lon = linha['lon']
    
    if not -90 <= float(lat) <= 90:
        if ',' not in str(lat):
            lat.insert(2, ',')
        else:
            print("id:", linha['id'])
            print("lat:", lat)
    if not -180 <= float(lon) <= 180:
        if ',' not in str(lon):
            lon.insert(2, ',')
        else:
            print("id:", linha['id'])
            print("lon:", lon)


3) Normaliza coluna lat com 5 casas decimais

In [ ]:
for col in ['lat', 'lon']:
    escolas_copia[col] = escolas_copia[col].astype(float).round(5)

#### Substitui abreviações de endereço

In [ ]:
# verifica possíveis abreviações
abreviacoes = set()

for i, linha_escola in escolas_copia.iterrows():
    lista_palavra_endereco = linha_escola['endereco'].split()
    for palavra in lista_palavra_endereco:
        if len(palavra) <= 4 and not palavra.isdigit():
            abreviacoes.update([palavra])

# verifica possíveis abreviações para logradouros
filtro_abreviacoes = [abreviacao for abreviacao in abreviacoes if abreviacao.startswith('R') or abreviacao.startswith('AV') or abreviacao.startswith('PCA')]
filtro_abreviacoes

In [ ]:
escolas_copia['endereco'] = escolas_copia['endereco'].str.replace('R. ', 'RUA ').replace('PCA. ', 'PRACA ', regex=True).replace('AV. ', 'AVENIDA ', regex=True).replace('AV ', 'AVENIDA ', regex=True)

Valida normalização de abreviações do campo endereco

In [ ]:
for i, linha_escola in escolas_copia.iterrows():
    if linha_escola['endereco'].startswith('R. ')  or \
       linha_escola['endereco'].startswith('AV. ') or \
       linha_escola['endereco'].startswith('AV ') or \
       linha_escola['endereco'].startswith('PCA. '):
        print(linha_escola['endereco'])

#### Normaliza os endereços

1) Normaliza endereços sem números

In [ ]:
escolas_copia['endereco'] = escolas_copia['endereco'].apply(lambda x: x.replace("/", "").replace(".", "").strip())

In [ ]:
# checa os valores de endereços sem números
enderecos_sem_numeros = set()

# separa enderecos em lista de listas
enderecos = [linha_escola['endereco'].split() for i, linha_escola in escolas_copia.iterrows()]

# verifica itens da lista que possuem valores SN (sem números)
for lista in enderecos:
    for item in lista:
        if item.startswith("SN") and item != "SN":
            enderecos_sem_numeros.update([item])

print(enderecos_sem_numeros)

In [ ]:
escolas_copia['endereco'] = escolas_copia['endereco'].str.replace('SNO', 'SN', regex=True).replace('SNDEG', 'SN', regex=True)

2) Normaliza campos com endereços incorretos

In [ ]:
escolas_copia[['id', 'endereco']].loc[escolas_copia['endereco'].str.count('RUA') > 1]

In [ ]:
escolas_copia[['id', 'endereco']].loc[escolas_copia['endereco'].str.count('AVENIDA') > 1]


In [ ]:
escolas_copia[['id', 'endereco']].loc[escolas_copia['endereco'].str.contains('AVENIDA') & escolas_copia['endereco'].str.contains('RUA')]

In [ ]:
escolas_copia['endereco'][escolas_copia['id'] == 144] = 'RUA LUISA MARILAC, 20'

In [ ]:
escolas_copia['endereco'][escolas_copia['id'] == 260] = 'RUA DA VITORIA'

#### Separa logradouro do número do endereço

1) Normaliza separador do logradouro e número como vírgula

In [ ]:
# Função para normalizar o endereço
def normaliza_endereco(endereco):
    # Verifica se o endereço contém um número
    if any(caracter.isdigit() and ' SN' not in endereco for caracter in endereco):
        
        # verifica indice da string que possui o número
        for caracter in endereco:
            if caracter.isdigit():
                indice = endereco.index(caracter)
                
                # verifica se tem vírgula
                if ',' not in endereco:
                    endereco = endereco[:indice-1] + ', ' + endereco[indice:]
    
    # verifica se contém SN
    elif ' SN' in endereco and not ',' in endereco:
        indice = endereco.index(' SN')
        endereco = endereco[:indice] + ', ' + endereco[indice+1:]
    
    return endereco


In [ ]:
# Aplica a função ao DataFrame
escolas_copia['endereco'] = escolas_copia['endereco'].apply(normaliza_endereco)

2) Cria campos logradouro e numero no DF

In [ ]:
escolas_copia[['logradouro', 'numero']] = escolas_copia['endereco'].str.split(',', expand=True)

3) Substitui números não informados no campo endereco

In [ ]:
escolas_copia['numero'].fillna('Não informado', inplace=True)

### Adicionar campo tipo de cada escola

1) Lista tipos possíveis de escola

In [ ]:
# listar tipos das escolas
inicio = list()
escolas_copia['escolas_postos'].apply(lambda x: inicio.append((x.split())[0]))
set(inicio)

2) Cria campo tipo no DF

In [ ]:
def mapear_tipo_escola(nome_da_escola: str):
    if nome_da_escola.startswith(("CIEP", "CENTRO")):
        return "CIEP"
    elif nome_da_escola.startswith("COLEGIO"):
        return "COLEGIO"
    elif nome_da_escola.replace(".", "").startswith(("ESCOLA", "EM")):
        return "EM"
    else:
        return None # Caso padrão, ou seja, nenhum valor encontrado

escolas_copia['tipo_escola'] = escolas_copia['escolas_postos'].apply(mapear_tipo_escola)

### Cria DF com dados completos de cada escola

In [ ]:
# aplica merge entre as tabelas
escolas_merge_1 = escolas_copia.merge(material_didatico_copia, how='inner', on='id')

In [ ]:
escolas_merge_2 = escolas_merge_1.merge(subprefeituras_copia, how='inner', left_on='bairro', right_on='nome')

### Normaliza coluna id (ex.: '024')

In [ ]:
escolas_merge_2['id'] = escolas_merge_2['id'].apply(lambda x: str(x).zfill(3))

### Ordena as colunas do DF

In [ ]:
# cria df final
colunas=['id_da_escola', 'nome_da_escola', 'tipo_da_escola', 'logradouro', 'numero', 'bairro', 'subprefeitura', 'latitude', 'longitude', 'quantidade_de_material']
df_final=pd.DataFrame(columns=colunas)

In [ ]:
# atribui valores às colunas do df final
df_final['id_da_escola']=escolas_merge_2['id']
df_final['nome_da_escola']=escolas_merge_2['escolas_postos']
df_final['tipo_da_escola']=escolas_merge_2['tipo_escola']
df_final['logradouro']=escolas_merge_2['logradouro']
df_final['numero']=escolas_merge_2['numero']
df_final['bairro']=escolas_merge_2['bairro']
df_final['subprefeitura']=escolas_merge_2['subprefeitura']
df_final['latitude']=escolas_merge_2['lat']
df_final['longitude']=escolas_merge_2['lon']
df_final['quantidade_de_material']=escolas_merge_2['quantidade']

### Ordenando as linhas utilizando o algoritmo do vizinho mais próximo

In [ ]:
from scipy.spatial.distance import cdist

# Crie uma matriz de distâncias entre todas as escolas
coordenadas = df_final[['longitude', 'latitude']].values
matriz_distancias = cdist(coordenadas, coordenadas, 'euclidean')

# Função para encontrar o caminho usando o algoritmo do vizinho mais próximo
def vizinho_mais_proximo(matriz_distancias):
    qtd_escolas = len(matriz_distancias)
    nao_visitadas = set(range(qtd_escolas))
    escola_atual = 0  # Comece da primeira escola
    caminho = [escola_atual]
    nao_visitadas.remove(escola_atual)

    while nao_visitadas:
        escola_mais_proxima = min(nao_visitadas, key=lambda x: matriz_distancias[escola_atual][x])
        caminho.append(escola_mais_proxima)
        escola_atual = escola_mais_proxima
        nao_visitadas.remove(escola_atual)

    return caminho

# Encontre o caminho usando o algoritmo do vizinho mais próximo
caminho_otimo = vizinho_mais_proximo(matriz_distancias)

# Reordene o DataFrame com base no caminho ótimo
df_final = df_final.iloc[caminho_otimo]

# Redefina os índices se necessário
df_final.reset_index(drop=True, inplace=True)

# df_final agora está reordenado com base em uma solução aproximada


### Escreve arquivo csv

In [ ]:
# escreve dataframe
df_final.to_csv("dados_tratados/dados_completos_escolas.csv", index=False)

### Contabiliza quantidade de material escolar a ser entregue por subprefeitura

In [ ]:
qnt_material_subpref = df_final[['subprefeitura', 'quantidade_de_material']].groupby(['subprefeitura']).sum().reset_index()

### Escreve arquivo csv

In [ ]:
qnt_material_subpref.to_csv("dados_tratados/qnt_material_subprefeitura.csv", index=False)